#Disclaimer:

Please try to understand the code skeleton before directly using this for the HW4P2. If you encounter anything that you are not able to understand or you feel is wrong do post on Piazza. 

#Updates:

1. In attention module check for the updated documentation. We should pass all the time-steps of Key and Value projection.
2. In method `train` look for the updated mask logic, current logic makes all the instances 1.
3. Check for the expected shape of the variable `text_input` in `train`. Though this is not a hard bound, but if you are getting a transpose shape than you need to change some other logic too. Hence we recommend you make sure the size of `text_input` is the same as mentioned.

**Mount Drive**

  People not using Colab can delete this block

In [2]:
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.nn.utils as utils
import pickle as pk
from torch.utils.data import DataLoader, Dataset 
import time

import torch.nn.utils.rnn as rnn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.nn.utils.rnn import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# **Load data**

Loading all the numpy files containing the utterance information and text information

In [3]:
speech_train = np.load('train_new.npy', allow_pickle=True, encoding='bytes')
speech_valid = np.load('dev_new.npy', allow_pickle=True, encoding='bytes')
speech_test = np.load('test_new.npy', allow_pickle=True, encoding='bytes')

transcript_train = np.load('./train_transcripts.npy', allow_pickle=True,encoding='bytes')
transcript_valid = np.load('./dev_transcripts.npy', allow_pickle=True,encoding='bytes')
print("Data Loading Sucessful.....")

Data Loading Sucessful.....


# **Transform Text Data**

`transform_letter_to_index` function transforms alphabetical input to numerical input. Each letter is replaced by its corresponding index from `letter_list` .

In [4]:
letter_list = ['<sos>','<eos>','A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q',\
             'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '-', "'", '.', '_', '+', ' ']

In [5]:
def transform_letter_to_index(transcript, letter_list):
    '''
    :param transcript :(N, ) Transcripts are the text input
    :param letter_list: Letter list defined above
    :return letter_to_index_list: Returns a list for all the transcript sentence to index
    '''
    i = 0
    letter_to_index_list = []
    for trans in transcript:
        sentence = ' '.join(list(np.array(trans, dtype='str')))
        sentence_index = []
        sentence_index.append(letter_list.index('<sos>') + 1)
        for letter in sentence:
            sentence_index.append(letter_list.index(letter) + 1)
            
        sentence_index.append(letter_list.index('<eos>') + 1)
        letter_to_index_list.append(np.array(sentence_index))   
    
    return np.array(letter_to_index_list)

In [6]:
character_text_train = transform_letter_to_index(transcript_train, letter_list)
character_text_valid = transform_letter_to_index(transcript_valid, letter_list)
print("Transformed data sucessfully.....")

Transformed data sucessfully.....


# **DataLoader**

Below is the dataloader for the homework.

*    You are expected to fill in the collate function if you use this code skeleton.

In [7]:
class Speech2Text_Dataset(Dataset):
  def __init__(self, speech, text=None, train=True):
    self.speech = speech
    self.train = train
    if(text is not None):
      self.text = text
  def __len__(self):
    return self.speech.shape[0]
  def __getitem__(self, index):
    if(self.train):
      return self.speech[index], self.text[index]
    else:
      return self.speech[index]

#.astype(np.float32)

In [8]:
def collate_train(batch_data):
  '''
  Complete this function.
  I usually return padded speech and text data, and length of 
  utterance and transcript from this function 
  '''

  inputs,targets = zip(*batch_data) 
  input_lens = [len(seq) for seq in inputs]
  target_lens = [len(seq) for seq in targets]
  inputs = [torch.tensor(l) for l in inputs]   
  inputs = pad_sequence(inputs)
  targets = [torch.tensor(l) for l in targets]
  targets = pad_sequence(targets,batch_first=True)
  return inputs.to(device), targets.to(device), torch.LongTensor(input_lens).to(device), torch.LongTensor(target_lens).to(device)

def collate_test(batch_data):
  '''
  Complete this function.
  I usually return padded speech and length of 
  utterance from this function 
  '''

  inputs = batch_data
  input_lens = [len(seq) for seq in inputs]
  inputs = [torch.tensor(l) for l in inputs]
  inputs = pad_sequence(inputs)
  return inputs.to(device), torch.LongTensor(input_lens).to(device)


In [9]:
Speech2Text_train_Dataset = Speech2Text_Dataset(speech_train, character_text_train)
Speech2Text_test_Dataset = Speech2Text_Dataset(speech_test, None, False)

train_loader = DataLoader(Speech2Text_train_Dataset, batch_size=128, shuffle=True, collate_fn=collate_train)

In [10]:
Speech2Text_valid_Dataset = Speech2Text_Dataset(speech_valid, character_text_valid)
Speech2Text_validT_Dataset = Speech2Text_Dataset(speech_train, None, False)


# **Pyramidal BiLSTM**
 

*   The length of utterance (speech input) can be hundereds to thousands of frames long.
*   Paper reports that that a direct LSTM implementation as Encoder resulted in slow convergence and inferior results even after extensive training.
*   The major reason is inability of `AttendAndSpell` operation to extract relevant information from a large number of input steps.

In [11]:
class pBLSTM(nn.Module):

  def __init__(self, input_dim, hidden_dim):
      super(pBLSTM, self).__init__()
      self.blstm = nn.LSTM(input_size=input_dim,hidden_size=hidden_dim,num_layers=1,bidirectional=True)
  def forward(self,lstm_outputs, short_lengths):
    '''
    :param x :(N,T) input to the pBLSTM
    :return output: (N,T,H) encoded sequence from pyramidal Bi-LSTM 
    '''
    
    out, lens = rnn.pad_packed_sequence(lstm_outputs)
    
    current_length = out.size()[0]
    
    if current_length%2!=0:
        current_length = current_length - 1
        
    out = out[:current_length, :, :]
    new_length = int(current_length/2)
    batch_size = out.size()[1]
    frequency = out.size()[2]
    out = out.transpose(0,1)
    lstm_input = out.reshape(batch_size,new_length,int(frequency*2))
    
    lstm_input = lstm_input.transpose(0,1)
    
    lstm_input_packed = utils.rnn.pack_padded_sequence(lstm_input, lengths=short_lengths, batch_first=False, enforce_sorted=False)
    
    outputs, _ = self.blstm(lstm_input_packed)
    
    return outputs

# **Encoder**

*    Encoder takes the utterances as inputs and returns the key and value.
*    Key and value are nothing but simple projections of the output from pBLSTM network.

In [12]:
class Encoder(nn.Module):
  def __init__(self, input_dim, hidden_dim, value_size=128,key_size=128):
    super(Encoder, self).__init__()
    self.lstm = nn.LSTM(input_size=input_dim,hidden_size=hidden_dim,num_layers=1,bidirectional=True)
    '''
    self.cnn1 = nn.Conv1d(in_channels=(hidden_dim*2), out_channels=(hidden_dim*2)*2, kernel_size=2, stride=2, padding=1)
    self.bn1 = nn.BatchNorm1d((hidden_dim*2)*2)
    self.relu1 = nn.ReLU6(inplace=True)
    
    self.cnn2 = nn.Conv1d(in_channels=(hidden_dim*2), out_channels=(hidden_dim*2)*2, kernel_size=2, stride=2, padding=1)
    self.bn2 = nn.BatchNorm1d((hidden_dim*2)*2)
    self.relu2 = nn.ReLU6(inplace=True)
    
    self.cnn3 = nn.Conv1d(in_channels=(hidden_dim*2), out_channels=(hidden_dim*2)*2, kernel_size=2, stride=2, padding=1)
    self.bn3 = nn.BatchNorm1d((hidden_dim*2)*2)
    self.relu3 = nn.ReLU6(inplace=True)
    '''
    
    #Here you need to define the blocks of pBLSTMs
    self.pbLSTM1 = pBLSTM((hidden_dim*2)*2, hidden_dim)
    self.pbLSTM2 = pBLSTM((hidden_dim*2)*2, hidden_dim)
    self.pbLSTM3 = pBLSTM((hidden_dim*2)*2, hidden_dim)
    
    self.key_network = nn.Linear(hidden_dim*2, value_size)
    self.value_network = nn.Linear(hidden_dim*2, key_size)
    self.dpo1 = nn.Dropout(p=0.2)
    self.dpo2 = nn.Dropout(p=0.2)
    self.dpo3 = nn.Dropout(p=0.2)
  
  def forward(self,x, lens, train=True):
    rnn_inp = utils.rnn.pack_padded_sequence(x, lengths=lens, batch_first=False, enforce_sorted=False)
    lstm_output, _ = self.lstm(rnn_inp)
    
    lens_np = lens.to('cpu').numpy()
    
    short_len1 = lens_np//2
    pbLSTM1_output = self.pbLSTM1(lstm_output, short_len1)
    
    if train==True:
        out, lensp = rnn.pad_packed_sequence(pbLSTM1_output)
        out = self.dpo1(out)
        pbLSTM1_output = utils.rnn.pack_padded_sequence(out, lengths=lensp, batch_first=False, enforce_sorted=False)

    
    short_len2 = lens_np//4
    pbLSTM2_output = self.pbLSTM2(pbLSTM1_output, short_len2)
    
    if train==True:
        out, lensq = rnn.pad_packed_sequence(pbLSTM2_output)
        out = self.dpo2(out)
        pbLSTM2_output = utils.rnn.pack_padded_sequence(out, lengths=lensq, batch_first=False, enforce_sorted=False)
    
    
    short_len3 = lens_np//8
    pbLSTM3_output = self.pbLSTM3(pbLSTM2_output, short_len3)
    
    if train==True:
        out, lensq = rnn.pad_packed_sequence(pbLSTM3_output)
        out = self.dpo3(out)
        pbLSTM3_output = utils.rnn.pack_padded_sequence(out, lengths=lensq, batch_first=False, enforce_sorted=False)

    #Use the outputs and pass it through the pBLSTM blocks

    linear_input, _ = utils.rnn.pad_packed_sequence(pbLSTM3_output)
    keys = self.key_network(linear_input)
    value = self.value_network(linear_input)

    return keys, value, short_len3


# **Attention**

*    Attention is calculated using key, value and query from Encoder and decoder.

Below are the set of operations you need to perform for computing attention.

```
energy = bmm(key, query)
attention = softmax(energy)
context = bmm(attention, value)
```



In [13]:
class Attention(nn.Module):
  def __init__(self):
    super(Attention, self).__init__()
    #self.smax = nn.Softmax(dim=0)
    
  def forward(self, query, key, value, lengths):
    '''
    :param query :(N,context_size) Query is the output of LSTMCell from Decoder
    :param key: (T,N,key_size) Key Projection from Encoder per time step
    :param value: (T,N,value_size) Value Projection from Encoder per time step
    :return output: Attended Context
    :return attention_mask: Attention mask that can be plotted  
    '''
    
    query = query.view(query.size()[0],query.size()[1],1)
    key = key.transpose(0,1)
    value = value.transpose(0,1) 
    energy = torch.bmm(key,query)
    #print(energy.size())
    energy = energy.transpose(1,2)

    mask = torch.arange(key.size(1)).unsqueeze(0) >= lengths.unsqueeze(1)
    mask = mask.to(device)
       
    energy = energy.squeeze(1)
    
    energy.masked_fill_(mask, -1e9)
    
    attention = nn.functional.softmax(energy, dim=1)
    
    context = torch.bmm(attention.unsqueeze(1), value)
    
    return context.view(context.size()[0],context.size()[2])
    

# **Decoder**

*    As mentioned in Recitation-9 each forward call of decoder deals with just one time step. Thus we use LSTMCell instead of LSLTM here.
*    Output from the second LSTMCell can be used as query here for attention module.
*    In place of `value` that we get from the attention, this can be replace by context we get from the attention.
*    Methods like Gumble noise and teacher forcing can also be incorporated for improving the performance.

In [14]:
from torch.distributions.gumbel import Gumbel

class Decoder(nn.Module):
  def __init__(self, vocab_size, hidden_dim, value_size=128, key_size=128,  isAttended=False):
    super(Decoder, self).__init__()
    self.embedding = nn.Embedding(vocab_size, hidden_dim)
    
    self.lstm1 = nn.LSTMCell(input_size=hidden_dim+value_size, hidden_size=hidden_dim)
    self.lstm2 = nn.LSTMCell(input_size=hidden_dim, hidden_size=key_size)
    self.dpo4 = nn.Dropout(p=0.2)
    self.dpo5 = nn.Dropout(p=0.2)
    self.isAttended = isAttended
    if(isAttended):
      self.attention = Attention()
    self.character_prob = nn.Linear(key_size+value_size,vocab_size)

  def forward(self, key, values, text=None, train=True, key_lengths = None, TFRate = 0.1):
    '''
    :param key :(T,N,key_size) Output of the Encoder Key projection layer
    :param values: (T,N,value_size) Output of the Encoder Value projection layer
    :param text: (N,text_len) Batch input of text with text_length
    :param train: Train or eval mode
    :return predictions: Returns the character perdiction probability 
    '''
    batch_size = key.shape[1]
    ##values_max_len = values.shape[0]
    if(train):
      max_len =  text.shape[1] - 1
      embeddings = self.embedding(text)
    else:
      max_len = 250
    
    predictions = []
    hidden_states = [None, None]
    ##prediction = torch.zeros([batch_size,1],dtype=torch.int32).to(device) + 33
    prediction = torch.ones(batch_size).to(device)
    
    gumbelRate = 0.1
    
    for i in range(max_len):
      '''
      Here you should implement Gumble noise and teacher forcing techniques
      '''
      '''
      if i < values_max_len:
        value_index = i
      else:
        value_index = (values_max_len - 1)
      '''
        
      teacher_forcing_choice= np.random.choice([0,1], p = [TFRate, (1-TFRate)])
    
      if(train):
        if teacher_forcing_choice == 1 or i==0:
            char_embed = embeddings[:,i,:]
        else:
            #char_embed = self.embedding(prediction.argmax(dim=-1))
            prediction_gumbel = Gumbel(prediction.to('cpu'), torch.FloatTensor([gumbelRate])).sample().to(device)
            #print(prediction_gumbel[0,:])
            #print(prediction_gumbel.argmax(dim=-1))
            char_embed = self.embedding(prediction_gumbel.argmax(dim=-1))
      else:
        if i == 0:
            char_embed = self.embedding(torch.LongTensor(prediction.tolist()).to(device))
        else:
            char_embed = self.embedding(prediction.argmax(dim=-1))

        
      
      #When attention is True you should replace the values[i,:,:] with the context you get from attention

      if i == 0:       
        context = values[i,:,:]
      
    
      #print(char_embed.size())
      #print(context.size())
      
      inp = torch.cat([char_embed,context], dim=1)
      

      hidden_states[0] = self.lstm1(inp,hidden_states[0])
    
    
      inp_2 = hidden_states[0][0]
        
      if train==True:
          inp_2 = self.dpo4(inp_2)
      
      
      hidden_states[1] = self.lstm2(inp_2,hidden_states[1])

      output = hidden_states[1][0]
        
      if train==True:
          output = self.dpo5(output)
        
      context = self.attention(output, key, values, torch.tensor(key_lengths))
        
      #print(output.size())
      #print(values[i,:,:].size())
      prediction = self.character_prob(torch.cat([output, context], dim=1))
      predictions.append(prediction.unsqueeze(1))

    return torch.cat(predictions, dim=1)

# **Sequence to Sequence Model**

*    We train an end-to-end sequence to sequence model comprising of Encoder and Decoder.

In [15]:
class Seq2Seq(nn.Module):
  def __init__(self,input_dim,vocab_size,hidden_dim,value_size=128, key_size=128,isAttended=False):
    super(Seq2Seq,self).__init__()

    self.encoder = Encoder(input_dim, hidden_dim)
    self.decoder = Decoder(vocab_size, hidden_dim*2, isAttended=True)
  def forward(self,speech_input, speech_len, text_input=None,train=True, TF_rate = 0.1):
    TRN = train
    key, value, key_lens = self.encoder(speech_input, speech_len,train=TRN)
    if(train):
      predictions = self.decoder(key, value, text_input, key_lengths=key_lens, TFRate = TF_rate)
    else:
      predictions = self.decoder(key, value, text=None, train=False, key_lengths=key_lens)
    return predictions


# **Learning**

Defining the Sequence to Sequence model, optimizer and criterion for learning.

Train routine is also provided here which can be referenced while writing validation and test routine.

In [18]:
def initialize(mdl):
    for name, parameter in mdl.named_parameters():
        if 'weight' in name:
            nn.init.xavier_normal_(parameter)

In [25]:
model = Seq2Seq(input_dim=40,vocab_size=(len(letter_list)+1),hidden_dim=256, isAttended=True)
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(reduce=False).to(device)
#model.apply(initialize)

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [26]:
'''
checkpoint = torch.load("model_params_dpo_new2_21.tar")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
'''

In [21]:
def train(model,train_loader, num_epochs, criterion, optimizer): 
  TF_Rate = 0.1
  for epochs in range(0, num_epochs):
    loss_sum = 0
    since = time.time()
    
    print("Epoch: ", epochs)
    
    
    for (batch_num, collate_output) in enumerate(train_loader):
      with torch.autograd.set_detect_anomaly(True):
        
        
        
        speech_input, text_input, speech_len, text_len = collate_output
        speech_input = speech_input.to(device)
        '''
        Please check if the text_input is of the (Batch_size, Sequence_length) i.e. (B,L)
        '''
        text_input = text_input.to(device)
        
        

        predictions = model(speech_input, speech_len ,text_input, TF_rate = TF_Rate)
        mask = torch.zeros(text_input.size()).to(device)

        for i in range(len(text_len)):
          mask[i,:text_len[i]] = 1

        mask = mask[:,1:]
        mask = mask.reshape(-1).to(device)
        

        predictions = predictions.contiguous().view(-1, predictions.size(-1))
        '''
        If you do not have text_input as (B,L) but have (L,B) instead then make
        sure that you uncomment the next line of code
        '''
        #text_input = text_input.T
        #print("text_input size")
        #print(text_input.size())
        text_input = text_input[:, 1:]
        text_input = text_input.contiguous().view(-1)

        #print(text_input)
        #print(predictions.argmax(dim=-1))
        #print(predictions)
        loss = criterion(predictions, text_input)
        masked_loss = torch.sum(loss*mask)
        
        #print(text_len.size()[0])
        
        masked_loss1 = masked_loss/(torch.sum(text_len) - text_len.size()[0])

        optimizer.zero_grad()
        masked_loss1.backward()

        torch.nn.utils.clip_grad_norm(model.parameters(), 2)
        
        optimizer.step()

        current_loss = float(masked_loss.item())/int(torch.sum(mask).item())

        if  batch_num % 25 == 1:
          print('train_loss', current_loss)
     
    if TF_Rate < 0.4:
        TF_Rate = TF_Rate * 1.05
    
    if epochs%1 == 0:
        torch.save({
                    'epoch': epochs,
                    'model_state_dict':model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': current_loss,
                    }, "model_params_dpo_new2_" + str(epochs) + ".tar")

In [22]:
train(model, train_loader, 26, criterion, optimizer)

Epoch:  0


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:60: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


train_loss 3.3444778707784106
train_loss 2.8553258489128037
train_loss 2.458418285389482
train_loss 2.3424324719599774
train_loss 2.0986038367080995
train_loss 2.001934452143978
train_loss 1.9493888300376647
train_loss 1.8832825651160146
Epoch:  1
train_loss 1.880969104398269
train_loss 1.8276460407250839
train_loss 1.7348645122719857
train_loss 1.7990073567560938
train_loss 1.6040909676823099
train_loss 1.5290259737216974
train_loss 1.3742976293431157
train_loss 1.259224964735999
Epoch:  2
train_loss 1.1833989160649316
train_loss 0.9738041902141582
train_loss 0.8546082894935562
train_loss 0.7954202451884723
train_loss 0.7443933981803919
train_loss 0.6872215291129458
train_loss 0.687012698251003
train_loss 0.6276096867919012
Epoch:  3
train_loss 0.6216111565505025
train_loss 0.6168176662322321
train_loss 0.5821829196960322
train_loss 0.521454969214684
train_loss 0.5099526328983245
train_loss 0.4398380815693021
train_loss 0.4716506169361755
train_loss 0.4661292662482806
Epoch:  4
train_

In [27]:
test_loader = DataLoader(Speech2Text_test_Dataset, batch_size=1, shuffle=False, collate_fn=collate_test)

In [34]:
for  (batch_num, collate_output) in enumerate(test_loader):
    #input_speech, x, input_length, y = collate_output
    input_speech, input_length = collate_output
    input_speech = input_speech.to(device)
    #print(input_speech.size())
    #print(input_length)
    #print(x)
    predictions = model(input_speech, input_length , train=False)
    #print(predictions[0,100,:])
    #print(predictions)
    out = torch.argmax(predictions, dim=-1)
    #print(out.size())
    letter_lis = np.array(letter_list)
    seq = "".join(letter_lis[out.cpu()-1].reshape(250,))
    
    if ('<eos>' in seq):
        EOS_index=seq.index('<eos>')
        seq = seq[:EOS_index]
    
    print(seq)
    #if batch_num == 10:
        #break
    

THE COMPANY ALSO DON'T ATED FIFTY THOUSAND DOLLARS TO A LOCAL ENVIRONMENTAL GROUP TO HIGHER SPECIALIST TO EVALUATE THE COMPANY'S PLANS TO CLEAN UP CONTAMINATED SCITES
MR. BENTAT'S COSF FROM MANDATORY TESTING HASN'T BEEN ENDORSED BY THE WHITE HOUSE PERIOD
A NUSURELY SISSURED YESTERDAY SAID THAT THE LINE OF CREDIT WOULD BE GUARANTEED BY PURROVIEN SOVER PERIOD
THE BUY OUT PLAN IS CONTENTIONED ON ALLEGHENING RECEIVING THAT WAVER PERIOD
HOWEVER INCREASING THE COST OF RESEARCH ANNIMALS SHOULD MOTIVE ATRE SEARCHURS NOT TO WASTE THEM ONE MERELY CURIOUS OR REPETTED A STUDIES
IN AUGUST COMMA A PACIFIC TELESIS GROUP UNIT MADE A SYMILAR GUARANTEE TO THE COMMUNICATIONS WORKERS OF AMERICA COMMA PROVIDING TRAINING FOR NEW POSITIONS OF NECESSARY PERIOD
AN AMBISHOS NINETEEN HYPHEN NATIONAL EFFORT TO HALT TWENTY YEARS OF WARRAN CAMBODIA ENDED WITHOUT AN AGREEMENT PERIOD
THE NEW CHAIRMAN KNOWS THAT HAS POLITICAL CONNECTIONS MAKE SOME MARKET PARTICIPANTS SUSPICIOUS THAT HE WALL TRIATEST DIMULATE THE ECONO

THE SHATTLED WHAT BE THE FIRST TO LIFE OPSINCE BENINETEEN EIGHTY SIX CHANGE TORE EXPLOSION
THEY VELUENT OF A BOOK DISTRIBUTED IN ANOTHER BURROW CAUGESTIEST DREAMS SCURT ABOUT LITTLE BOY WHO CHALLENGES SEXES AND BY DRESSING AS A GERRAL
YESTERDAY'S ANNOUNCEMENT BY REPRESENTATIVE HOWARD MEANS TO HOUSE WILL VOTE ON THE STUMM IS A MED MATS SEPARATELY FROM THE OVERALL CONFRONT SURPORT PERIOD
AND SOME CHAINS COMMA SUCH AS HOW DAY CORPORATION COMMA SHARED IN CORPORATION AT HIGHT HOTELS CORPORATION COMMA INSISTLY WILL MAKE PLANNI OF ROMS AVAILABLE AT BARGAIN RATES PERIOD
THE CORDING TO HIGH SLASH LOW RESEARCH COMMA THERE TEY FIVE MILLION PEOPLE PURCHASED A FISHING LISENS IN NINETEEN EIGHTY FOUR PERIOD
MITCHELL HELD AN ECONOMIST AT SNIFT BARNY HARRIS UPUMAN AND COMPANY THINKS MR. GREENSPAN WANTS TO ESTABLIFHE HIS CRIDENTIALS AS AN INFLATION FIGHTER
THE NEW CHAIRMAN NOS THAT HIS POLITICAL CONNACTIONS MAKE SOME MARKET PRRTICIPANTS SUSPICIOUS THAT EABLE TRY TO STIMILATE THE ECONOMY TO HELP G. O. P.

U. S. EXPORTS ROSE IN A MONTH BUT NOT NEARLY AS MUCH AS IMPORTS
IF IT IS AN ONE THYING IT'S ANOTHER
DOUBLE-QUOTE I TOTLEM IS THE INDIVIDIALOGIES COMMA DOUBLE-QUOTE SAYS MR. AMERAVE COMMA A PARTY VETERAN PERIOD
THE DEADLINE FOR DOING SO IS FEBRUARY TWENTY FIRST TWO MONTHS BEFOREGELLETTS APRIL TWENTY FIRST ANNUAL MEETING
THE COMPANY ALSO DONETED FIFTY THOUSAND DOLLARS TO A LOCAL ENVIRONMENTAL GROUP TO HIGHER SPECIALISTS TO EVALUATE THE COMPANY'S PLANS TO CLEAN UP A CONTAMINATED SITES
BUT THERE'S AN IGLAY SIDE TO WHAT'S GOING OUT IN LOCAL GOVERNMENT THAT DISERVES CLOSED A RATENTION
HE SAID HE WOULD REOPEN NATIONAL ASSEBLY DEBATE ON CONSTITUTIONAL REVISION COMMA WHICH AN APRIL HE ITS SUSPENDED INTO AFTER THE NINETEEN EIGHTY EIGHT OLYMPICS PERIOD
MR. HOLMES A COURT SAID HE PLANS TO REVIEW THE STRUCT CHURIVE BELL GROUP HER STATE
AT FORTY YEARS OLD MR. WEING AS RELATIVELY YOUNG FOR HIS JOB AND AT UPSETS COLLEAGUES WHO ARE USED TO GRADEAL AND PROMOTION SYSTEM BASED ON SENIOR
NUMEROUS COMMARDS 

WATCHING A COMPUTER HYPHEN DRIVEN LATED TURN RAW METAL INTO A PRECISION PART COMMA MR. AMONN CONCEDES THE RISK PERIOD
SO MR. WRINGTTELS PEOPLE USE FIFTY
THE RESULT WAS PORIED
IT IS A PONETION COMMA AT OFTEN DISAPPOINTING EXPERIOUS PERIOD
AND THEY MAY BE AVERALLET GET AWAY WITH A LOT MORE OF THESE PUBLICITY MAN NEW VERSE WITHOUT GIVING UP MUCH REAL GROUND
SO FINALLY THE WORRIED NEW YORKER HAD AS BLOWD TESTED FOR EVIDENCE OF THE AGE VIRUS
THE OTHER IS THAT WE SHOULD HAVE THE SURPLUS BUT WE SHOULD HAVE THE SURPLUS BUT WEAL THE SAID
DOUBLE-QUOTE IF THE SOVIETS TO SIDE TO CHANGE THE NATURE OF THE MISSION COMMA THEY NOW HAVE THE RIGHT KIE HERE COMMA DOUBLE-QUOTE SAYS THE ISRAELI OFFICIAL PERIOD
THE MONEY TO PUT THIS PROPOSITION ON THE BALLOT CAME PARTLY FROM THE GRAND HYPHEN CHAMPION GIVER COMMA A SOCIATION PACK PERIOD
IN NINETEEN EIGHTY COMMA THE FLANES RESUMED HOME GAMES PERIOD
THE SHUTTLE WOULD BE THE FIRST TO LIFT OFF SENCE THE NINETEEN EIGHTY SEX CHALL INTERE EXPLOSION PERIOD
EVENTY ABO

AS MS. KENDAL AND MR. LOWECCIITE SOUTH AFFER CAS SENTRAL GOVERNMENT IS WINCO A BIG NOMBERING TANK
THE FICIALS AND TO GOES THE GAUGH A SAID THE PLANE IN NORD REPUSSED TO LAND AFTER ENTERING HUNDRED AND EARLIER PERIOD
A SPOKESMAN FOR THE NUCLEAR REGULATORY COMMISSIONS THAT THREE M. DECIDED TO RECALL ADDITIONAL MODELS OF ITS AIR I. ANDIZERS AFTER RADIATION LEAKS WERE DISCOVERED AT FOUR FOCILITIES USING THOSE MODELS PERIOD
THE DOUBLE-QUOTE HEAD COUNT COMMA DOUBLE-QUOTE A BOGUS MEASURING DIVICE IF EVERTHER WAS ONE COMMA REMAINED THE SAME SEMI-COLON THERE WAS NO INCREASE IN SALARIED PERSONAL PERIOD
JORD'S E. R. CANAIAR THE SECOND WAS NAMED TO THE NEW POST OF SENIOR VICE PRESIDENT IN CHARGE OF LONG RAINED PLANNING IN ERLATED GOVERNMENT RELATIONS
THE CITY CAN PROBABLY HANDLE TWO PROGRAMS COMMA SAYS SHARP MA MORIL ADMINISTRATOR CHARLESCOKE COMMA DOUBLE-QUOTE BUT THREE OR FOUR COULD BE PROBLEMATIC PERIOD DOUBLE-QUOTE
BASED ON ALL THAT WE KNOW COMMA AND AT VICE OF HOR COSSE COMMA RENOTE NO REASON

THE RATING CONCERN ALSO PLACED THE SINGLE A TWO SENIOR DEBT RATING OF C. S. O. S. X. S. EX T TEXAS GAS TRENSMISSION UNIT ON REVIEW WITH DIRECTION ON CERTAIN
SOME SOURCES SAID SOME OF THE LESS TRADITIONAL HOLDERS OF GENERALE DUA BELGEES SHARES COULD BE TEMPTED BY THE PROSPECT OF CAPITAL GAINS
THE DUTAC INCORPORATED'S SUPPORTERS COULDN'T CONTAIN THEM SOUVES PERIOD
BUT IT'S DIFFICULT TO SEE WHERE THE COMPANY GOES FROM HERE SAID
THE PROBLEM CONTINUES TO BE MONNOTNY AND FASHION COMMA SAID CURRENT BARKETINED COMMA THE PULISHER OF THE RETAIL MARKETING REPORT COMMA AND INDUSTRY NEESLETTER PERIOD
MOST IMPORTANTLY COMMA THE BOOK COMMA LIKELY EXCIBITE COMMA PRESENTS IN ENGAGING SURVEY OF INTERLACED FORMS TO THE CENTURIES DASH ONE WEEKEND LEARN FROM AND A JOY PERIOD
THE TWO COMPANIES AGREED TO SHETTLE A CASE FOR THREE BILLION DOLLARS IN LATE DECEMBER
I. B. M. FELL ONE AND SUBBONN AIDS TO ONE HUNDRED TWENTY AND THREE EIGHTHS ON MORE THAN TWO POINT FIVE MILLION SHARES
BUT ATTEMPTS TIES LATE A MISSIN

THE COMBINED BUSINESSES ARE COUNTED FOR TWO HUNDRED THIRTY FIVE MILLION DOLLARS OF DRAVOS EIGHT HUNDRED NINETY THREE MINUION DOLLARS IN NINETEEN EIGHTY FIVE REPUNIOU
HE ARGUES THAT FRIDAY'S UNEMPLOYMENT FIGURES UNDERMINED THE THESES OF O SHARPLY SLOWING ECONOMY
SO YEEN FUTURES PRICES WERE MIXED WOULD BEINS AN OIL OF FINISHING HIGHER
BUT ACCORDS CREDIBILITIES HAS A ROADED TO THE POINT THAT IF THEY PAN AMD DIRETTERS THAT WALKED RINTERVACKER THEY WANT GET THOSE CONCESSIONS
EVEN THE STATE HIGHERS PRIVATE REPAIRMAN PERIOD
IT'S ABOUT LIKE SEVENTEEN DAYS IN A BAB BE'S LIEE COMMA DOUBLE-QUOTE SAYS MR. DICKY COMMA A VETER AN INGUARS TEACHER HERE A JAPAN PERIOD
THEIGER LOANS AND LOWER TAXES ARE DROWN FIFTEEN THOUSAND WORKERS A YEAR INTO LEGALIZED PRIVATE SERVICES PERIOD
THEY HAVE BEEN IN THE U. S. SNCH THE EARLY NINETEEN SEVENTIES PERIOD
BECOUSE THEY CONSERVATIVE PARTED CONTROLS THE PARLIANA THE WINNER OF THE TOP PARTY POST ASSO BECOMES PRIME MINISTER
EDWARD'S TENSIS SLACKS OF RALLED WAS MOLL CO